In [18]:
from qdrant_client import QdrantClient
from qdrant_client.http.models import Distance, VectorParams, PointStruct

client = QdrantClient(path="qdrant_db") # or QdrantClient(":memory:")

In [38]:
# connect sqlite3
import sqlite3
import pandas as pd
path_sqlite3 = "/Users/qxnam/Documents/semeter/DATA-PLATFORM/IUHKart/iuhkart/db.sqlite3"
conn = sqlite3.connect(path_sqlite3)

query_promt = '''
SELECT d.slug, d.product_name, c.category_name
FROM product d 
JOIN category c ON c.category_id = d.category_id
'''

df = pd.read_sql_query(query_promt, conn)
df.head()
# content = df.iloc[0].to_dict()
# content

,slug,product_name,category_name
0,women-canvas-messenger-bag-youth-ladies-fashio...,Women Canvas Messenger Bag Youth Ladies Fashio...,Luggages & Bags
1,100-genuine-leather-crossbody-bag-for-women-sh...,100% Genuine Leather Crossbody Bag For Women S...,Luggages & Bags
2,10pcs-candy-color-bear-pendant-charms-for-neck...,10Pcs Candy Color Bear Pendant Charms for Neck...,Jewelry & Accessories
3,juya-18k-real-gold-plated-new-creativetree-of-...,Juya 18K Real Gold Plated New CreativeTree Of ...,Jewelry & Accessories
4,2023-new-trend-fashion-four-leaf-grass-printin...,2023 New Trend Fashion Four Leaf Grass Printin...,Luggages & Bags


In [19]:
# init collection
client.create_collection(
    collection_name="product",
    vectors_config=VectorParams(size=384, distance=Distance.COSINE),
)

True

In [34]:
# embedding slug into qdrant with 5 record head
import requests, uuid

for _, it in df.head().iterrows():
    content = it.to_dict()
    text_embedding = content['slug']
    rep = requests.get(f'http://9net.ddns.net:9008/embedding?q={text_embedding}')
    vector = rep.json()['embedding']
    point = PointStruct(id=str(uuid.uuid4()),
                      vector=vector,
                      payload=content)
    client.upsert(collection_name="product", points=[point])

In [39]:
# search
prompt = 'women'
prompt_embedding = requests.get(f'http://9net.ddns.net:9008/embedding?q={prompt}').json()['embedding']
results = client.search(collection_name="product", query_vector=prompt_embedding, limit=2)
results[0].model_dump()

{'id': '13926ef6-5fd7-4e11-bd1a-46d91393f772',
 'version': 0,
 'score': 0.4602108656855013,
 'payload': {'slug': 'women-canvas-messenger-bag-youth-ladies-fashion-shoulder-bag-student-large-capacity-female-crossbody-bags-woman-packet',
  'product_name': 'Women Canvas Messenger Bag Youth Ladies Fashion Shoulder Bag Student Large Capacity Female Crossbody Bags Woman Packet',
  'category_name': 'Luggages & Bags'},
 'vector': None,
 'shard_key': None}

In [40]:
# close
client.close()